In [2]:
import pandas as pd

# Read merged_features.csv
features_df = pd.read_csv('../../result/merged_features.csv')

# Read merged_labelled.csv
labelled_df = pd.read_csv('../../result/merged_labelled.csv')

In [3]:
data = pd.merge(features_df, labelled_df[['name', 'label']], on='name', how='left')
data.to_csv('../../merged_features_with_label.csv', index=False)

In [2]:
import os 
import pandas as pd 
import numpy as np
import torch
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, make_scorer
from sklearn.utils import resample
from joblib import dump, load
import pickle

csv_path = "/data4/shine/WebGraph/result/merged_features_with_label.csv"

os.getcwd()
data = pd.read_csv(csv_path)
#data = data.sample(frac=1) #row shuffle
# print(type(data))
# print(data.head())

nCar = data.shape[0]
nVar = data.shape[1]
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

X = data.drop("CLASS", axis=1)
y = data["CLASS"]

count_class_0, count_class_1 = data['CLASS'].value_counts()
print("-----")
print("NonAD (before dropping) : {}".format(count_class_0))
print("AD (before dropping) : {}".format(count_class_1))

print("-----")
print("NonAD (after dropping) : {}".format(count_class_0))
print("AD (after dropping) : {}".format(count_class_1))
print("Ratio : {:.1f} : {:.1f}".format(count_class_0 / (count_class_0 + count_class_1) * 100, count_class_1 / (count_class_0 + count_class_1) * 100))




: 

: 

In [3]:
# Define the stratified cross-validation object
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

to_exclude = ["DOMAIN_NAME", "NODE_ID", "FEATURE_KATZ_CENTRALITY", "FEATURE_FIRST_PARENT_KATZ_CENTRALITY", "FEATURE_SECOND_PARENT_KATZ_CENTRALITY"]

accuracy_history = []
precision_history = []
recall_history = []

# For each fold
for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    
    print("Fold ", fold)
    if fold != 0:
        continue
    
    fold_train_y = y[train_index]
    fold_test_y = y[test_index]
    
    for class_label in np.unique(y):
        train_count = np.count_nonzero(fold_train_y == class_label)
        test_count = np.count_nonzero(fold_test_y == class_label)
        if fold == 0 or fold == 5 or fold == 9:
            print("Class label:", class_label, "train count:", train_count, "test count:", test_count)

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    X_train = X_train.drop(to_exclude, axis=1)
    X_test = X_test.drop(to_exclude, axis=1)

    # Define classifier
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    # Train
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)

    accuracy_history.append(accuracy_score(y_test, y_pred))
    precision_history.append(precision_score(y_test, y_pred))
    recall_history.append(recall_score(y_test, y_pred))

# Print
print("Average score:", np.mean(accuracy_history))
print("Average precision:", np.mean(precision_history))
print("Average recall:", np.mean(recall_history))

# Save
with open('../saved_model/rf_model_origperc.pt', 'wb') as f:
    pickle.dump(clf, f)

Fold  0
Class label: 0 train count: 337613 test count: 37513
Class label: 1 train count: 51433 test count: 5715
Fold  1
Fold  2
Fold  3
Fold  4
Fold  5
Fold  6
Fold  7
Fold  8
Fold  9
Average score: 0.9603729064495234
Average precision: 0.8997203355972833
Average recall: 0.7881014873140857


In [10]:
# SELECT QUERIES
# variables
query_size = 130000

data2 = pd.read_csv(csv_path + csv_name)
# data2 = data2.sample(frac=1) #row shuffle
X = data2.drop("CLASS", axis=1)
y = data2["CLASS"]

# concate
X = X.drop(to_exclude, axis=1)

y_result = clf.predict(X)
y_result = pd.DataFrame(y_result, columns=["CLASS"])
y_label = pd.DataFrame(y, columns=["CLASS"])
instances_concat = pd.concat([X, y_result], axis=1)
remaining_concat = pd.concat([X, y_label], axis=1)

instances_concat.to_csv("/home/shine/PoisoningAttack/dataset/temp_org_for_surrogate.csv", index=False)
remaining_concat.to_csv("/home/shine/PoisoningAttack/dataset/temp_org_for_inversion.csv", index=False)


In [12]:
selcted_query = pd.read_csv(csv_path + "preprocessed_temp_org_for_surrogate.csv")
remaining_queries = pd.read_csv(csv_path + "preprocessed_temp_org_for_inversion.csv")

# count "all" instances
if query_size == "all":
    query_size = len(instances_concat)
    print("all query size : {}".format(query_size))

# Select random N queries
selcted_query = selcted_query.sample(n=query_size)

# save
if query_size == len(instances_concat):
    query_size = "all"
print(selcted_query.shape[1])
selcted_query.to_csv("/home/shine/PoisoningAttack/dataset/for_surrogate/query_origperc_{}.csv".format(query_size), index=False)

# save remaining query
selected_indices = selcted_query.index
print(selected_indices)
remaining_queries = remaining_queries.drop(selected_indices)
remaining_queries = remaining_queries[remaining_queries["CLASS"] == 1]
print(remaining_queries.shape[1])
remaining_queries.to_csv("/home/shine/PoisoningAttack/dataset/origperc_{}_for_inversion.csv".format(query_size), index=False)



321
Int64Index([345816,  11419, 158916,  59153,  25011,  30022, 166354, 212005,
            222572, 406669,
            ...
            205380, 227173, 396351, 400624,   9949, 214255, 196664, 108121,
            393368, 144911],
           dtype='int64', length=130000)
321


In [31]:
import pandas as pd

csv_path = "/home/shine/PoisoningAttack/dataset/"

# csv_name_ad = "origperc_130000_for_inversion.csv"
# csv_name_pgd = "perturbed_features_pgd.csv"
csv_name_nbackdoor = "reordered_bin_processed_perturbed_features_nbackdoor_bin_original_perc0.3_eps4.0.csv"

# data_ad = pd.read_csv(csv_path + csv_name_ad)
# data_pgd = pd.read_csv(csv_path + csv_name_pgd)
data_nbackdoor = pd.read_csv(csv_path + csv_name_nbackdoor)

# X_ad = data_ad.drop('CLASS', axis=1)
# feature_columns_pgd = list(data_pgd.columns.difference(['CLASS']))
X_nbackdoor = data_nbackdoor.drop('CLASS', axis=1)

# y_ad = data_ad[['CLASS']]
# y_pgd = after_mapping_target_pgd
y_nbackdoor = data_nbackdoor[['CLASS']]
 
# predict_ad = clf.predict(X_ad)
# predict_pgd = clf.predict(X_pgd)
predict_nbackdoor = clf.predict(X_nbackdoor)

# print(y_ad, predict_ad)
# print(accuracy_score(y_pgd, predict_pgd))
print(accuracy_score(y_nbackdoor, predict_nbackdoor))


0.3316536221007112


In [38]:
perturbed_csv_path = "/home/shine/PoisoningAttack/dataset/features_rf_only_cat2.csv"
df = pd.read_csv(perturbed_csv_path)
to_exclude = ["DOMAIN_NAME", "NODE_ID", "FEATURE_KATZ_CENTRALITY", "FEATURE_FIRST_PARENT_KATZ_CENTRALITY", "FEATURE_SECOND_PARENT_KATZ_CENTRALITY"]

df = df.drop(to_exclude, axis=1)
y_pred = clf.predict(df)

print(y_pred)